In [73]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [74]:
import pandas as pd
import os
from sqlalchemy import create_engine

os.environ["API_DB_USER"] = "postgres"
os.environ["API_DB_PASSWORD"] = "ywngtpwyBH0922"
os.environ["API_DB_HOST"] = "localhost"
os.environ["API_DB_PORT"] = "5431"
os.environ["API_DB_DB"] = "rugby4cast"

def get_connection():
    if "API_DB_USER" in os.environ:
        user = os.getenv("API_DB_USER")
        password = os.getenv("API_DB_PASSWORD")
        host = os.getenv("API_DB_HOST")
        port = os.getenv("API_DB_PORT")
        database = os.getenv("API_DB_DB")
    else:
        raise EnvironmentError("No Env Vars Supplied to Access DB")

    connection_str = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    return create_engine(
        connection_str, pool_pre_ping=True, pool_recycle=600
    )

def get_session(query):
    engine = get_connection()
    # session_made = sessionmaker(bind=engine)
    # session = session_made()
    # session.execute(query)
    data = pd.read_sql_query(query, engine)

    return data

In [75]:
LEAGUES = '''
    ('Coppa Italia',
    'Coppa Italia Serie C, Girone A',
    'Coppa Italia Serie C, Girone B',
    'Coppa Italia Serie C, Girone C',
    'Coppa Italia Serie C, Girone D',
    'Coppa Italia Serie C, Girone E',
    'Coppa Italia Serie C, Girone F',
    'Coppa Italia Serie C, Girone G',
    'Coppa Italia Serie C, Girone H',
    'Coppa Italia Serie C, Girone I',
    'Coppa Italia Serie C, Girone L',
    'Coppa Italia Serie C, Girone M',
    'Coppa Italia Serie C, Knockout stage',
    'Coppa Italia Serie D',
    'Italian Coppa Italia',
    'Italian Serie A',
    'Italian Serie B',
    'Italian Supercoppa',
    'Primavera Cup, Knockout stage',
    'Serie A',
    'Serie B',
    'Serie B, Promotion Playoffs',
    'Serie B, Relegation Playoffs',
    'Serie C, Girone A',
    'Serie C, Girone A, Relegation Playoffs',
    'Serie C, Girone B',
    'Serie C, Girone B, Relegation Playoffs',
    'Serie C, Girone C',
    'Serie C, Girone C, Relegation Playoffs',
    'Serie C, Promotion Playoffs',
    'Serie C, Relegation Playoffs',
    'Serie D, Girone A',
    'Serie D, Girone A, Playoffs',
    'Serie D, Girone B',
    'Serie D, Girone B, Playoffs',
    'Serie D, Girone C',
    'Serie D, Girone C, Playoffs',
    'Serie D, Girone D',
    'Serie D, Girone D, Playoffs',
    'Serie D, Girone E',
    'Serie D, Girone E, Playoffs',
    'Serie D, Girone F',
    'Serie D, Girone F, Playoffs',
    'Serie D, Girone G',
    'Serie D, Girone G, Playoffs',
    'Serie D, Girone H',
    'Serie D, Girone H, Playoffs',
    'Serie D, Girone H, Playout',
    'Serie D, Girone I',
    'Serie D, Girone I, Playoffs',
    'Serie D, Girone I, Playout',
    'Serie D, Poule Scudetto, Group 2',
    'Serie D, Poule Scudetto, Group 3',
    'Serie D, Poule Scudetto, Knockout stage',
    'Supercoppa',
    'Supercoppa Serie')
'''


matches_query = f' SELECT * FROM football_matches WHERE league IN {LEAGUES}'

odds_query = f'SELECT * FROM football_betting fb WHERE fb.match_id in (select fm.match_id from football_matches fm where ("league" in {LEAGUES}))'

TABLE_NAME_PRED = "football_matches"

data = get_session(matches_query)
odds = get_session(odds_query)
print(data.shape)
print(odds.shape)

(37472, 90)
(369520, 11)


In [76]:
data.head()

,pt1,pt2,score_pt1,score_pt2,ground,country,league,date,time,season,...,crosses_pt2,dribbles_completed_pt2,dribbles_pt2,possession_lost_pt2,duels_won_pt2,aerials_won_pt2,tackles_pt2,interceptions_pt2,clearances_pt2,attendance
0,milan,cagliari,4,1,,,Serie A,2011-05-14 18:45:00+00:00,19:45:00,2011,...,0,0,0,0,0,0,0,0,0,NaN
1,fiorentina,bologna,1,1,,,Serie A,2011-05-15 10:30:00+00:00,11:30:00,2011,...,0,0,0,0,0,0,0,0,0,NaN
2,lazio,genoa,4,2,,,Serie A,2011-05-14 16:00:00+00:00,17:00:00,2011,...,0,0,0,0,0,0,0,0,0,NaN
3,torino,triestina,2,0,,,Serie B,2011-05-14 13:00:00+00:00,14:00:00,2011,...,0,0,0,0,0,0,0,0,0,NaN
4,ascoli,siena,3,2,,,Serie B,2011-05-13 18:45:00+00:00,19:45:00,2011,...,0,0,0,0,0,0,0,0,0,NaN


In [77]:
odds.head()

,team,link,match_id,bookie,date,odds,hodds,hcap,gw,collected_at,sport
0,pistoiese,https://www.betmonitor.com/odds-comparison/foo...,lucchese_pistoiese_2022-04-23,volcanobet,2022-04-23 17:30:00+00:00,0.298507,0.0,0.0,2022-04-18 17:30:00+00:00,2022-04-23 03:00:00+00:00,football
1,lucchese,https://www.betmonitor.com/odds-comparison/foo...,lucchese_pistoiese_2022-04-23,evrotip,2022-04-23 17:30:00+00:00,0.454545,0.0,0.0,2022-04-18 17:30:00+00:00,2022-04-23 03:00:00+00:00,football
2,lucchese,https://www.betmonitor.com/odds-comparison/foo...,lucchese_pistoiese_2022-04-23,victory,2022-04-23 17:30:00+00:00,0.454545,0.0,0.0,2022-04-18 17:30:00+00:00,2022-04-23 03:00:00+00:00,football
3,lucchese,https://www.betmonitor.com/odds-comparison/foo...,lucchese_pistoiese_2022-04-23,ps3838,2022-04-23 17:30:00+00:00,0.462963,0.0,0.0,2022-04-18 17:30:00+00:00,2022-04-23 03:00:00+00:00,football
4,lucchese,https://www.betmonitor.com/odds-comparison/foo...,lucchese_pistoiese_2022-04-23,wwin,2022-04-23 17:30:00+00:00,0.462963,0.0,0.0,2022-04-18 17:30:00+00:00,2022-04-23 03:00:00+00:00,football


In [78]:
odds['match_id'].nunique()

1997

In [79]:
odds['date'] = pd.to_datetime(odds['date'])
odds.sort_values(by='date')

,team,link,match_id,bookie,date,odds,hodds,hcap,gw,collected_at,sport
70348,draw,https://oddspedia.com/api/v1/football/spezia-p...,pordenone_calcio_spezia_2021-08-13,netbet-it,2021-08-13 15:45:00+00:00,0.312500,0.000000,0.0,2021-08-09 15:45:00+00:00,2010-01-01 00:00:00+00:00,football
70288,spezia,https://oddspedia.com/api/v1/football/spezia-p...,pordenone_calcio_spezia_2021-08-13,parimatch,2021-08-13 15:45:00+00:00,0.400000,0.578035,0.0,2021-08-09 15:45:00+00:00,2010-01-01 00:00:00+00:00,football
70289,spezia,https://oddspedia.com/api/v1/football/spezia-p...,pordenone_calcio_spezia_2021-08-13,eurobet,2021-08-13 15:45:00+00:00,0.425532,0.000000,0.0,2021-08-09 15:45:00+00:00,2010-01-01 00:00:00+00:00,football
70290,spezia,https://oddspedia.com/api/v1/football/spezia-p...,pordenone_calcio_spezia_2021-08-13,betathome,2021-08-13 15:45:00+00:00,0.409836,0.000000,0.0,2021-08-09 15:45:00+00:00,2010-01-01 00:00:00+00:00,football
70291,spezia,https://oddspedia.com/api/v1/football/spezia-p...,pordenone_calcio_spezia_2021-08-13,sts,2021-08-13 15:45:00+00:00,0.390625,0.552486,0.0,2021-08-09 15:45:00+00:00,2010-01-01 00:00:00+00:00,football
...,...,...,...,...,...,...,...,...,...,...,...
365838,juventus,https://www.betmonitor.com/odds-comparison/foo...,fiorentina_juventus_2023-02-12,apuestatotal,2023-02-12 18:00:00+00:00,0.523560,0.000000,0.0,2023-02-06 18:00:00+00:00,2023-02-11 12:00:00+00:00,football
365839,juventus,https://www.betmonitor.com/odds-comparison/foo...,fiorentina_juventus_2023-02-12,germania,2023-02-12 18:00:00+00:00,0.540541,0.000000,0.0,2023-02-06 18:00:00+00:00,2023-02-11 12:00:00+00:00,football
365840,juventus,https://www.betmonitor.com/odds-comparison/foo...,fiorentina_juventus_2023-02-12,tab sport,2023-02-12 18:00:00+00:00,0.555556,0.000000,0.0,2023-02-06 18:00:00+00:00,2023-02-11 12:00:00+00:00,football
365812,juventus,https://www.betmonitor.com/odds-comparison/foo...,fiorentina_juventus_2023-02-12,victory,2023-02-12 18:00:00+00:00,0.534759,0.000000,0.0,2023-02-06 18:00:00+00:00,2023-02-11 12:00:00+00:00,football


In [31]:
def group_odds(df):
    return df.groupby(by=['match_id', 'team'])

In [32]:
def get_average_odds(df):
    odds_group = group_odds(df)
    return odds_group.mean().reset_index()

In [33]:
odds = get_average_odds(odds)

/var/folders/bl/xbg_536n2_n123s38z4hxbx00000gn/T/ipykernel_6823/4063377542.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  return odds_group.mean().reset_index()


In [47]:
def rename_team_to_pt1(df):
    return df.rename(columns={'team': 'pt1'})

In [34]:
def cut_odds(df):
    return df[['match_id', 'team', 'odds']]

In [48]:
def format_odds(df):
    df = cut_odds(df)
    df = rename_team_to_pt1(df)
    return df

In [49]:
def index_match_id_and_team(df, team_col):
    df = df.set_index(['match_id', team_col])
    return df

In [50]:
def standardise_indexes(df1, df1_col, df2, df2_col):
    df1 = index_match_id_and_team(df1, df1_col)
    df2 = index_match_id_and_team(df2, df2_col)
    return df1, df2

In [54]:
def join_odds_to_data(odds_df, data_df):
    odds_df = format_odds(odds_df)
    odds_df, data_df = standardise_indexes(odds_df, 'pt1', data_df, 'pt1')
    combined = data_df.merge(odds_df, on=['match_id', 'pt1'])
    return combined.reset_index()

In [55]:
combined = join_odds_to_data(odds, data)

In [56]:
combined.head()

,match_id,pt1,pt2,score_pt1,score_pt2,ground,country,league,date,time,...,dribbles_completed_pt2,dribbles_pt2,possession_lost_pt2,duels_won_pt2,aerials_won_pt2,tackles_pt2,interceptions_pt2,clearances_pt2,attendance,odds
0,lazio_udinese_2022-01-18,lazio,udinese,1,0,Stadio Olimpico,Italy,Coppa Italia,2022-01-18 16:30:00+00:00,16:30:00,...,12,26,179,71,12,29,16,15,NaN,0.600078
1,sassuolo_torino_2022-01-23,torino,sassuolo,1,1,Stadio Olimpico Grande Torino,Italy,Serie A,2022-01-23 14:00:00+00:00,14:00:00,...,0,3,145,52,22,12,6,15,NaN,0.510058
2,messina_virtus_francavilla_2022-01-23,virtus_francavilla,messina,5,1,Stadio Giovanni Paolo II,Italy,"Serie C, Girone C",2022-01-23 16:30:00+00:00,16:30:00,...,0,0,0,0,0,0,0,0,NaN,0.654277
3,pistoiese_vis_pesaro_2022-01-30,pistoiese,vis_pesaro,0,1,Stadio Marcello Melani,Italy,"Serie C, Girone B",2022-01-30 13:30:00+00:00,13:30:00,...,0,0,0,0,0,0,0,0,NaN,0.399706
4,pescara_virtus_entella_2022-02-17,pescara,virtus_entella,0,0,Stadio Adriatico,Italy,"Serie C, Girone B",2022-02-17 20:00:00+00:00,20:00:00,...,0,0,0,0,0,0,0,0,NaN,0.412595


In [67]:
def check_arbitrage_opps(df):
    grouped = df.groupby('match_id').sum()
    arb_opps = grouped[grouped['odds']>1]
    print(f"There are {arb_opps.shape[0]:d} arbitrage opportunities.")

In [68]:
check_arbitrage_opps(combined)

There are 0 arbitrage opportunities.


/var/folders/bl/xbg_536n2_n123s38z4hxbx00000gn/T/ipykernel_6823/4255908011.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = df.groupby('match_id').sum()


In [69]:
combined.shape

(1997, 91)